In [18]:
from langchain_community.utilities import SQLDatabase
import os

In [19]:
%pwd

'c:\\Users\\adminn\\Documents\\GitHub\\AI-Database-Agent'

In [6]:
# os.chdir('..//')

In [20]:
db_path="data"+"/sqldb.db"
db=SQLDatabase.from_uri(f"sqlite:///{db_path}")
print(db.get_usable_table_names)

db.run(
    " SELECT * FROM Employee LIMIT 10;"
)

<bound method SQLDatabase.get_usable_table_names of <langchain_community.utilities.sql_database.SQLDatabase object at 0x000001EE93DDE8B0>>


"[(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com'), (2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com'), (3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com'), (4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com'), (5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00', '7727B 41 Ave', 'Calgar

# Q&A DataBase

In [30]:
import warnings
import openai
warnings.filterwarnings('ignore')
from dotenv import load_dotenv


In [ ]:
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
import os

open_api_key=os.getenv('OPENAI_API_KEY')

llm=ChatOpenAI(
    model='gpt-4',
    temperature=0,
    openai_api_key=open_api_key
)


In [17]:
llm.invoke('what is meaning of life?')

AIMessage(content="The meaning of life is a philosophical and metaphysical question related to the purpose or significance of life or existence in general. This concept can be interpreted in many different ways depending on one's beliefs. Some people believe the meaning of life is to pursue happiness or personal fulfillment, while others believe it is to contribute to the betterment of humanity, seek knowledge, or fulfill certain duties or responsibilities. In religious contexts, the meaning of life is often believed to be a divine purpose or plan. Ultimately, the answer can be deeply personal and subjective.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 13, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4', 'system_fingerprint': None,

In [24]:
from langchain.chains import create_sql_query_chain
chain=create_sql_query_chain(llm,db)
response=chain.invoke({'question':'how many artistis are there?'})
response

'SELECT COUNT("ArtistId") FROM "Artist"'

In [27]:
from langchain.chains import create_sql_query_chain
chain=create_sql_query_chain(llm,db)
response=chain.invoke({'question':'give me all column names that i have in  employee table?'})
response

'PRAGMA table_info(Employee);'

In [25]:
db.run(response)

'[(275,)]'

In [28]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query=create_sql_query_chain(llm,db)
excute_query=QuerySQLDataBaseTool(db=db)
chain=write_query | excute_query

chain.invoke({"question":'how many employees are there'})

'[(8,)]'

In [29]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt=PromptTemplate.from_template(

    """ Given the following user question,corresponding SQL query and SQL result and ansswer the user question
    
    Question: {question}
    SQL Query: {query}
    SQL Result: {result}
    Answer:"""
)

answer=prompt|llm|StrOutputParser()
chain=(
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | excute_query
    ) | answer
)

chain.invoke({"question":'give me the columns in employee table'})


'The columns in the employee table are EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, and Email.'